# Wav2Vec2 transfer learning on wav2vec2-base


In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import random
from transformers import Wav2Vec2Processor, Trainer, TrainingArguments, Wav2Vec2ForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

c:\Users\profi\OneDrive\Desktop\AI-Project--Speech-Emotion-Recognition\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [3]:
print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cu121
True


## Load Dataset


In [4]:
df_train = pd.read_csv('../data/train_dataset.csv')
df_train = df_train[['Filepath', 'Emotion']]
df_val = pd.read_csv('../data/val_dataset.csv')
df_val = df_val[['Filepath', 'Emotion']]
df_test = pd.read_csv('../data/test_dataset.csv')
df_test = df_test[['Filepath', 'Emotion']]

In [5]:
df_train

,Filepath,Emotion
0,./dataset/mlend\MLEndSND_Public\11093.wav,Question
1,./dataset/crema-d\AudioWAV\1011_TAI_ANG_XX.wav,Anger
2,./dataset/mlend\MLEndSND_Public\27019.wav,Neutral
3,./dataset/mlend\MLEndSND_Public\23420.wav,Bored
4,./dataset/crema-d\AudioWAV\1077_ITH_ANG_XX.wav,Anger
...,...,...
44216,./dataset/meld\train\dia873_utt1.mp4,Neutral
44217,./dataset/esd\0015\Happy\0015_000807.wav,Happy
44218,./dataset/mlend\MLEndSND_Public\40154.wav,Neutral
44219,./dataset/esd\0017\Sad\0017_001347.wav,Sad


In [6]:
df_val

,Filepath,Emotion
0,./dataset/esd\0020\Sad\0020_001395.wav,Sad
1,./dataset/meld\train\dia930_utt5.mp4,Neutral
2,./dataset/mlend\MLEndSND_Public\24481.wav,Bored
3,./dataset/crema-d\AudioWAV\1002_IEO_SAD_HI.wav,Sad
4,./dataset/esd\0011\Angry\0011_000373.wav,Anger
...,...,...
9471,./dataset/tess\YAF_disgust\YAF_take_disgust.wav,Disgust
9472,./dataset/mlend\MLEndSND_Public\43418.wav,Bored
9473,./dataset/mlend\MLEndSND_Public\02459.wav,Bored
9474,./dataset/mlend\MLEndSND_Public\10609.wav,Question


In [7]:
df_test

,Filepath,Emotion
0,./dataset/mlend\MLEndSND_Public\01620.wav,Neutral
1,./dataset/jl-corpus/Raw JL corpus (unchecked a...,Sad
2,./dataset/mlend\MLEndSND_Public\04997.wav,Question
3,./dataset/esd\0018\Surprise\0018_001454.wav,Surprise
4,./dataset/mlend\MLEndSND_Public\05521.wav,Neutral
...,...,...
9472,./dataset/tess\YAF_pleasant_surprised\YAF_half...,Surprise
9473,./dataset/esd\0019\Angry\0019_000651.wav,Anger
9474,./dataset/mlend\MLEndSND_Public\21688.wav,Neutral
9475,./dataset/esd\0020\Angry\0020_000377.wav,Anger


In [8]:
# Convert labels to integers
unique_labels = sorted(df_train['Emotion'].unique())
label_map = {label: idx for idx, label in enumerate(unique_labels)}
print(label_map)

df_train['Emotion'] = df_train['Emotion'].map(label_map)
df_val['Emotion'] = df_val['Emotion'].map(label_map)
df_test['Emotion'] = df_test['Emotion'].map(label_map)

{'Anger': 0, 'Bored': 1, 'Disgust': 2, 'Fear': 3, 'Happy': 4, 'Neutral': 5, 'Question': 6, 'Sad': 7, 'Surprise': 8}


In [9]:
df_train

,Filepath,Emotion
0,./dataset/mlend\MLEndSND_Public\11093.wav,6
1,./dataset/crema-d\AudioWAV\1011_TAI_ANG_XX.wav,0
2,./dataset/mlend\MLEndSND_Public\27019.wav,5
3,./dataset/mlend\MLEndSND_Public\23420.wav,1
4,./dataset/crema-d\AudioWAV\1077_ITH_ANG_XX.wav,0
...,...,...
44216,./dataset/meld\train\dia873_utt1.mp4,5
44217,./dataset/esd\0015\Happy\0015_000807.wav,4
44218,./dataset/mlend\MLEndSND_Public\40154.wav,5
44219,./dataset/esd\0017\Sad\0017_001347.wav,7


In [10]:
df_val

,Filepath,Emotion
0,./dataset/esd\0020\Sad\0020_001395.wav,7
1,./dataset/meld\train\dia930_utt5.mp4,5
2,./dataset/mlend\MLEndSND_Public\24481.wav,1
3,./dataset/crema-d\AudioWAV\1002_IEO_SAD_HI.wav,7
4,./dataset/esd\0011\Angry\0011_000373.wav,0
...,...,...
9471,./dataset/tess\YAF_disgust\YAF_take_disgust.wav,2
9472,./dataset/mlend\MLEndSND_Public\43418.wav,1
9473,./dataset/mlend\MLEndSND_Public\02459.wav,1
9474,./dataset/mlend\MLEndSND_Public\10609.wav,6


In [11]:
df_test

,Filepath,Emotion
0,./dataset/mlend\MLEndSND_Public\01620.wav,5
1,./dataset/jl-corpus/Raw JL corpus (unchecked a...,7
2,./dataset/mlend\MLEndSND_Public\04997.wav,6
3,./dataset/esd\0018\Surprise\0018_001454.wav,8
4,./dataset/mlend\MLEndSND_Public\05521.wav,5
...,...,...
9472,./dataset/tess\YAF_pleasant_surprised\YAF_half...,8
9473,./dataset/esd\0019\Angry\0019_000651.wav,0
9474,./dataset/mlend\MLEndSND_Public\21688.wav,5
9475,./dataset/esd\0020\Angry\0020_000377.wav,0


In [12]:
model_name = 'facebook/wav2vec2-base'
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    model_name, num_labels=len(label_map))

c:\Users\profi\OneDrive\Desktop\AI-Project--Speech-Emotion-Recognition\myenv\lib\site-packages\transformers\configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from emotion_datasets.SpeechEmotionDataset import SpeechEmotionDataset

In [14]:
# Load the dataset
train_dataset = SpeechEmotionDataset(df_train, processor)
val_dataset = SpeechEmotionDataset(df_val, processor)
test_dataset = SpeechEmotionDataset(df_test, processor)

In [15]:
train_dataset[0]

Keyword argument `truncate` is not a valid argument for this processor and will be ignored.


{'input_values': tensor([0.0288, 0.0384, 0.0252,  ..., 0.0002, 0.0002, 0.0002]),
 'labels': tensor(6)}

## Set Training Arguments


In [16]:
train_args = TrainingArguments(
    output_dir='./models/wav2vec2-base',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
    dataloader_num_workers=8,
    dataloader_pin_memory=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to=[]
)

c:\Users\profi\OneDrive\Desktop\AI-Project--Speech-Emotion-Recognition\myenv\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# Create function for computing metrics
def compute_metrics(pred):
    labels = pred.label_ids  # original labels
    preds = np.argmax(pred.predictions, axis=1)  # model predicted labels
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [18]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.692800,0.631136,0.785352,0.800488,0.785352,0.779182
2,0.512100,0.503121,0.831469,0.838243,0.831469,0.831971
3,0.422000,0.471603,0.848670,0.851782,0.848670,0.847472
4,0.363400,0.448454,0.859645,0.866218,0.859645,0.858771
5,0.302100,0.483887,0.867138,0.869256,0.867138,0.866469
6,0.273100,0.513195,0.867033,0.870502,0.867033,0.866512
7,0.226300,0.541591,0.867455,0.867228,0.867455,0.866628
8,0.170600,0.572830,0.875369,0.876897,0.875369,0.875096
9,0.143300,0.642123,0.872837,0.873308,0.872837,0.872029
10,0.120400,0.686484,0.873681,0.873697,0.873681,0.873088


TrainOutput(global_step=27640, training_loss=0.3547521937600783, metrics={'train_runtime': 4280.5408, 'train_samples_per_second': 103.307, 'train_steps_per_second': 6.457, 'total_flos': 1.6058954044680966e+19, 'train_loss': 0.3547521937600783, 'epoch': 10.0})

In [20]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.5728297233581543, 'eval_accuracy': 0.8753693541578725, 'eval_precision': 0.8768965102031938, 'eval_recall': 0.8753693541578725, 'eval_f1': 0.8750956067901096, 'eval_runtime': 36.7411, 'eval_samples_per_second': 257.913, 'eval_steps_per_second': 16.14, 'epoch': 10.0}


## Test Predictions


In [21]:
# Get model predictions on the test dataset
predictions = trainer.predict(test_dataset)
# Convert logits to predicted class labels
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids  # Ground truth labels

# Compute the confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=label_map.keys(), yticklabels=label_map.keys())

plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')

plt.xticks(rotation=45)
plt.yticks(rotation=45)

plt.tight_layout()
plt.show()